# Predictores de Saltos

Como hemos visto en el esquema pipeline, varias instrucciones son puestas en el pipeline, logrando de que en cada ciclo del procesador se lleven a cabo las diferentes etapas de cada instruccion. 

Ciclos|C1|C2|C3|C4|C5|C6|C7
-|-|-|
Instruccion 1|IF|ID|EX|MEM|WB
Instruccion 2|-|IF|ID|EX|MEM|WB
Instruccion 3|-|-|IF|ID|EX|MEM|WB

Esto se puede hacer en el caso de que se conozca con seguridad el orden de las instrucciones. El orden se conoce a en la mayoria de las instrucciones, pero no se lo conoce en las instrucciones de salto condicional. Esas instrucciones dependiendo se si se cumple o no la condicion setean en diferentes lugares el program counter. Y por lo tanto como no conocemos cual sera la siguiente instruccion no la podemos colocar en el pipeline. Entonces pensariamos que el procesador haria lo siguiente:

Ciclos|C1|C2|C3|C4|C5|C6|C7
-|-|-|
Instruccion 1|IF|ID|EX|MEM|WB
Instruccion 2|-|IF|ID|EX|MEM|WB
Instruccion Salto|-|-|IF|ID|EX|MEM|WB
Instruccion Siguiente|-|-|-|Stall|Stall|Stall|Stall|IF|ID|EX|MEM|WB

Perdiendo de esta forma 4 ciclos del procesador. Esto obviamente es poco performante, por lo tanto. Lo que comunmente se usa es un <b>Predictor de Saltos</b>. Que como su nombre lo dice intenta predecir a donde seguira el program counter. Y por lo tanto colocara las instrucciones que corresponden a su decision. Para que se vallan ejecutando. Si la prediccion fue correcta las deja, pero si resulta que la prediccion fue incorrecta debera sacar estas instrucciones.

Existen dos clases de predictores de saltos, los que realizan prediccion <b>estatica</b> y los que realizan prediccion <b>dinamica</b>. La diferencia radica en el momento en el que se realiza la prediccion, en el caso de la prediccion estatica se realiza en tiempo de <b>compilacion</b>, mientras que la dinamica se realiza en tiempo de <b>ejecucion</b>.

## Estrategias de Prediccion Estatica

- <b>Predecir todos los saltos como tomados</b>: Es tecnica es sencilla, pero su precision es bastante pobre.
- <b>Predicciones basadas en el codigo de operacion</b>: Esta basada en estudios que muestras que segun el tipo de operacion la probabilidad de salto cambia, se observa que para cierto codigos de operacion la probabilidad de salto es del orden del 80%-90%.
- <b>Predecir los saltos en funcion de su direccion</b>: Por ejemplo, los saltos <b>"hacia atras"</b> predecirlos como tomamos y los saltos <b>"hacia adelante"</b> predecirlos como no tomados. Esta tecnica esta basada en el hecho de que una gran maoria de los saltos hacia atras corresponden a bucles, y por lo tanto seran tomados todas las veces que el bucle se ejecute menos una. En cambio los saltos hacia adelante corresponden mas a estructuras if-then-else.

## Estrategias de Prediccion Dinamica

En estas tecnicas, la prediccion sobre el resultado de un salto se basa en informacion conocida solo en tiempo de ejecucion.

Dos estructuras son necesarias para realizar una prediccion dinamica:
- <b>Branch History Table (BHT)</b>: Es una tabla donde se guarda informacion sobre las ultimas ejecuciones de los saltos. Osea si los saltos ejecutados fueron tomados o no. A partir de esta informacion se predice si el siguiente salto sera tomado o no.
- <b>Branch Target Address Cache (BTAC)</b>: Es una tabla donde se almacena la direccion destino de los ultimos saltos ejecutados. 

### Branch Target Buffer

Estas dos estrucutras se combinan formando la base de todo predictor dinamico, que es el denominado <b>Branch Target Buffer</b>. Este es una pequenia memoria asociativa que guarda las direcciones de los ultimos saltos ejecutados asi como su destino. A su vez guarda informacion que permite predecir si el salto sera tomado o no.

Asi que cuando se hace el fetch de la instruccion se mira si la direccion de la instruccion esta en la <b>BTB</b>. Si es asi se miran los bits de prediccion y se decide si el salto ha de ser tomado o no. Si el salto no es tomado o la direccion no esta en la BTB en el siguiente ciclo se hace fetch de la siguiente instruccion en orden.

En cuanto a la politica de entrada de saltos en el BTB, <b>Perleberg y Smith</b> proponen su trabajo la siguiente premisa: un salto que no tenga potencial para mejorar el rendimiento, no sera introducido en el BTB. Para ello se pueden utilizar dos politicas: introducir un salto cuando es ejecutado por primera vez, o introducir un salto cuando es tomado. En su trabajo Perleberg concluye diciendo que para un BTB con un grado de asociativadad 4 el segundo metodo es mas preciso (tambien es el que mas se ajusta a su premisa). La premisa introducida por Perleberg para sacar saltos del BTB es: cuando sea necesario sacar un salto del BTB, descartar el que tenga menos potencial para mejorar el rendimiento. Bajo esta premisa el algoritmo propuesto seria <b>LRU (Least Recentyle Used)</b>, es decir sacar del BTB el salto que haga mas tiempo que no ha sido referenciado. Tambien  

<b>Calder</b> en su trabajo realiza una optimizacion sobre lo anteriormente propuesto que seria instroducir en el BTB solo los saltos tomados. De esta manera, cuando un salto no esta en el BTB se predice como no tomado. A su vez cuando un salto predicho como tomado realmente no es tomado, se invalida su entrada en el BTB.

Branch Address|History Bits|Target Address
-|-|-
-|-|-
-|-|-
-|-|-
-|-|-
-|-|-
-|-|-
-|-|-

Un punto clave en el disenio del BTB es el <b>algoritmo de prediccion</b>, tanto por el numero de bits empleados como por la forma de decidir la prediccion y actualizar la informacion. El metodo mas utilizado es el de un <b>contador saturado</b> de dos bits. Saturado quiere decir que cuando esta en su  valor maximo y lo incrementamos no pasa a cero, sino que se queda igual, a su vez cuando vale cero y lo decrementamos, su valor continua siendo cero.

El mecanismo de funcionamiento de este contador es el siguiente: cada vez que se tiene que hacer una prediccion se mira el bit de mas peso para decidir si el salto es tomado o no, y una vez que el salto es ejecutado se actualiza la informacion del contador de tal manera que si el salto ha sido tomado se incrementa y si el salto no ha sido tomado se decrementa. Esto significa que para valores altos del contador el sato predicho como tomado, mientras que para valores bajos sera predicho como no tomado.

El BTB es una buena estructura para prediccion de saltos, ya que aunque no consigue porcentajes de acierto espectaculares, su coste en hardware es poco comparado con otros predictores.

### Predictores basados en dos niveles de historia

Debido a la importancia de la predicion de saltos en el disenio de los computadores, ha resultado que en el BTB basado en un nivel de prediccion (contador saturado <b>up-down</b> que registra la historia de cada salto) no consigue un porcentaje de aciertos suficiente para las necesidades de los procesadores.

Diversos autores han propuesto predictores mas costosos a nivel de hardware que basan su predicion en dos niveles de historia: en un primer nivel recogen la historia de los n ultimos saltos ejecutados o de las n ultimas ejecucioens de un salto concreto (dependiendo de la implementacion). Con esta informacion, en un segundo nivel, se indexa una tabla de patrones que en cada entrada guarda una maquina de estados (normalmente codificada en dos bits) con la que se decide si el salto es tomado o no.


<b>Yeh y Patt</b>, presentan una primera aproximacion consistente en dos estructuras principales: <b>Branch History Register Table (BHRT)</b> y <b>Branch History Pattern Table (BHPT)</b>.

La <b>BHRT</b> es una tabla que contiene un registro de la historia de m bits para cada salto. Este registro es de desplazamiento y almacena la historia de las ultimas m ejecuciones de dicho salto. Por ejemplo, considerando T como salto tomado y N como salto no tomado, si un salto cualquiera ha tenido el siguiente comportamiento en sus ultimas 5 ejecuciones, TNTTN su registro de historia correspondiente sera 10110. A la codificacion binaria de este comportamiento se le llama <b>patron</b>.

La <b>BHPT</b> es una tabla de $2^m$ entradas, indexada por el registro de historia. Esta tabla guarda la historia de las ultimas ejecuciones de todos los saltos que tienen el mismo patron, representado por el registro historia.

Cuando llega un nuevo salto <b>$B_i$</b>, se obtiene su registro de historia de la <b>BHRT</b>, el contenido del cual es utilizado para indexar la Pattern Table, obteniendo <b>$S_c$</b>, que es la historia de la ejecucion siguiente del salto cuando tenia el patron dado porque el registr historia. Sobre estos bits es aplicada la funcion de prediccion $Z_c$, la cual nos dice si el salto va a ser tomado o no.

<img src="Imagenes/esquema.png">